# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846781746598                   -0.70    4.5
  2   -7.852305779154       -2.26       -1.53    1.0
  3   -7.852646187906       -3.47       -2.52    3.2
  4   -7.852646677360       -6.31       -3.38    2.2
  5   -7.852646686260       -8.05       -4.65    1.8
  6   -7.852646686727       -9.33       -5.60    3.0
  7   -7.852646686730      -11.59       -6.26    1.8
  8   -7.852646686730      -13.09       -7.63    2.0
  9   -7.852646686730      -14.75       -8.04    2.5
 10   -7.852646686730   +    -Inf       -9.38    3.0
 11   -7.852646686730      -14.75      -10.44    2.5
 12   -7.852646686730   +  -14.75      -10.87    1.8
 13   -7.852646686730   +  -15.05      -10.26    1.0
 14   -7.852646686730      -15.05       -9.71    2.0
 15   -7.852646686730   +  -14.57       -9.37    2.5
 16   -7.852646686730      -14.57       -9.26    1.2
 17   -7.852646686730   +  -15.05       -9.29 

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846796792479                   -0.70           4.5
  2   -7.852529498164       -2.24       -1.63   0.80    2.2
  3   -7.852637137522       -3.97       -2.71   0.80    1.0
  4   -7.852646549008       -5.03       -3.29   0.80    2.0
  5   -7.852646678648       -6.89       -4.16   0.80    1.5
  6   -7.852646686483       -8.11       -4.85   0.80    1.8
  7   -7.852646686722       -9.62       -5.76   0.80    2.0
  8   -7.852646686730      -11.11       -6.86   0.80    1.8
  9   -7.852646686730      -12.94       -7.68   0.80    2.5
 10   -7.852646686730   +    -Inf       -8.29   0.80    1.8
 11   -7.852646686730      -15.05       -9.01   0.80    1.5
 12   -7.852646686730   +  -15.05      -10.16   0.80    2.2
 13   -7.852646686730      -14.75      -10.69   0.80    2.5
 14   -7.852646686730   +  -14.75      -11.16   0.80    1.5
 15   -7.852646686730   +    -Inf      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.379921e+01     2.926761e+00
 * time: 0.4574751853942871
     1     1.494926e+00     1.863559e+00
 * time: 0.690040111541748
     2    -1.365442e+00     2.265997e+00
 * time: 0.7180361747741699
     3    -3.881186e+00     1.848818e+00
 * time: 0.7578921318054199
     4    -5.117198e+00     1.666880e+00
 * time: 0.7979521751403809
     5    -6.852429e+00     8.951877e-01
 * time: 0.8376140594482422
     6    -6.875273e+00     1.102213e+00
 * time: 0.8653721809387207
     7    -7.372344e+00     8.070090e-01
 * time: 0.8930351734161377
     8    -7.405682e+00     8.554857e-01
 * time: 0.9210031032562256
     9    -7.667032e+00     4.648194e-01
 * time: 0.9488570690155029
    10    -7.744921e+00     2.644307e-01
 * time: 0.9765892028808594
    11    -7.798751e+00     1.046200e-01
 * time: 1.0041592121124268
    12    -7.842115e+00     7.256412e-02
 * time: 1.0324389934539795
    13    -7.849709e+00     3.922563e-02
 * time: 1.06050109863

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846888029639                   -0.70    4.5
  2   -7.852326976629       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686714                   -2.55
  2   -7.852646686730      -10.81       -6.03
  3   -7.852646686730   +    -Inf      -12.82


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 9.155502641413414e-13
|ρ_newton - ρ_scfv| = 1.302996343344456e-13
|ρ_newton - ρ_dm|   = 1.1964223068345413e-9
